In [1]:
# ... import the required librairies


import seaborn as sns # Samuel Norman "Sam" Seaborn is a fictional character portrayed by Rob Lowe on the television serial drama The West Wing. Hence: sns
import pandas as pd
import datetime as dt

In [2]:
# ... import the required .csv file. Change the name to the file you prefer to look at. 

df= pd.read_csv('data/raw_data_2.csv',skipinitialspace=True)
df.head()

,genetic,length,mass,exercise,smoking,alcohol,sugar,lifespan
0,73.9,185,99.7,0.9,0.0,2.4,6.9,73.1
1,86.0,172,105.4,1.8,8.1,0.4,4.2,85.0
2,83.3,176,111.4,1.1,0.8,4.6,7.5,81.6
3,82.8,164,111.4,4.7,11.8,1.0,2.9,81.0
4,78.7,178,71.6,1.5,8.3,4.9,5.5,75.0


In [4]:
df.describe()

,smoking,lifespan
count,4096.000000,4096.000000
mean,9.807446,80.127783
std,6.921461,8.042270
min,0.000000,-1.000000
25%,3.300000,75.000000
50%,9.500000,79.700000
75%,15.900000,85.200000
max,22.200000,105.100000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4096 entries, 0 to 4095
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   genetic   4096 non-null   object 
 1   length    4096 non-null   object 
 2   mass      4096 non-null   object 
 3   exercise  4096 non-null   object 
 4   smoking   4096 non-null   float64
 5   alcohol   4096 non-null   object 
 6   sugar     4096 non-null   object 
 7   lifespan  4096 non-null   float64
dtypes: float64(2), object(6)
memory usage: 256.1+ KB


In [6]:

# ... removes turns non-int/non-flo to NaN and removes NaN

df['genetic'] = pd.to_numeric(df['genetic'], errors='coerce')
df['length'] = pd.to_numeric(df['length'], errors='coerce')
df['mass'] = pd.to_numeric(df['mass'], errors='coerce')
df['exercise'] = pd.to_numeric(df['exercise'], errors='coerce')
df['alcohol'] = pd.to_numeric(df['alcohol'], errors='coerce')
df['sugar'] = pd.to_numeric(df['sugar'], errors='coerce')


df.astype('float64').dtypes



genetic     float64
length      float64
mass        float64
exercise    float64
smoking     float64
alcohol     float64
sugar       float64
lifespan    float64
dtype: object

In [ ]:
# ## ... not used atm 
# # ... maak een column bmi en een column bmi_category om te kijken of dat impact heeft.  

# df['bmi'] = df['mass']/(df['length']/100)**2
# df.head()

In [ ]:

# ... select value to be looked at.

key = df.keys()

pipe = input('please type the name of the column you want to look at')
if pipe in key:
    print (f'We will be looking at "{pipe}" this time.')
else:
    print ('Please rerun this field and suply a valid column key.')

In [ ]:
# # ... get a loc  not used atm
# dx=df.columns.get_loc(pipe)
# print (dx)

In [ ]:
print(df.head())
print(df.describe())

In [ ]:
# ... test plot not used atm

# got any outliers?
# sns.boxplot(x=df['lifespan'])

# sns.boxplot(df[pipe])


#... alles buiten de lijnen is een outlier. 

In [ ]:
# ... safe the data from this notebook as a csv in the folder output 

df.to_csv('../data/output_data/data_{}_{}.csv'.format(pipe,dt.datetime.now().strftime("%Y-%m-%d %H-%M")), index=False,sep=';')

In [ ]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

<h3> Step 0: Data Sanitation </h3>
<li> null values (Nan), duplicates, object -> float/int</li>

In [ ]:
df = df.copy()
df.isnull().sum() #why is it missing, is it random? : input (impute) or delete? (some decisions come later outlier analysis, but some can be taken now)
df.duplicated().sum()
# dupli_rows = df[df.duplicated()]
# df.drop.duplicates() 
# check for special characters ??/>, delete and convert to int/float etc.

<h3>Step 1: Simply Look at the Data and state what you already can see</h3>

In [ ]:
df.type
df.describe()  #mean,sd, min,max

In [ ]:
#jointplot with distribution and regression line
sns.jointplot(data=df, x="alcohol", y="lifespan",marginal_kws=dict(bins=35), kind='reg')

<li>Decision: imputing do we detect strange outliers in the data do we impute or delete?</li>

<li> Quick Overall Graphical Overview (!warning, takes ~2min or more)</li>

In [ ]:
g = sns.PairGrid(df)
g.map(sns.lineplot)

<h3> Step 2: Zoom in on individual features </h3>
<li> Outliers </li>
<li> Strange data points </li>
<li> look for correlations: linear/log/etc.</li>
<li> Is the feature normally distributed </li>

In [ ]:
#Show scatter, boxplot a
fig, axes = plt.subplots(1,2,figsize=(15,5))
sns.stripplot(df, x='alcohol', y='lifespan',size=1, color=".8",ax=axes[0])
sns.boxplot(df[['alcohol']],ax=axes[1])

In [ ]:
#Show lineplot and/or jointplot, check for correlation (linear, positive/negative)
sns.lineplot(df,x='alcohol',y='lifespan')

In [ ]:
sns.jointplot(df, x="alcohol", y="lifespan",marginal_kws=dict(bins=35), kind='reg')

In [ ]:
#Show Q-Q plot and draw conclusion on linearity and if linear regression is applicable 
from seaborn_qqplot import pplot
myplot = pplot(df, x="lifespan", y="alcohol", kind='qq', height=5)

<h3> Step 4: Correlations of independent variables with dependent variable lifespan</h3>

__[Correlation Coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient)__

$ \rho_{x,y} = \frac{cov(x,y)}{\sigma_{x}\sigma{y}}$



In [ ]:
#colorfull matrix, showing correlations
df_corr = df[['lifespan', 'genetic', 
      #'length', 'mass', 
      'exercise', 'smoking',
       'alcohol', 'sugar', 'BMI']].dropna().corr()
df_corr.style.background_gradient(cmap='RdBu')

In [ ]:
#graphical correlation matrix view
sns.heatmap(df_corr,annot=True)

<li>Calculate P-values</li>

If p<0.05 (almost 0) the correlation for the feature is extremely likely to happen again if we collect more sample data and thus representative for the entire set


In [ ]:
r,p = stats.pearsonr(df.lifespan,df.smoking)
print('smoking corr:',round(r,4))
print('smoking p-val:',round(p,4))
r,p = stats.pearsonr(df.lifespan,df.exercise)
print('exercise corr:',round(r,4))
print('exercise p-val:',round(p,4))
r,p = stats.pearsonr(df.lifespan,df.genetic) #capital sensitive .<columnname>
print('BMI corr:',round(r,4))
print('BMI p-val:',round(p,4))

<h3>Step 5: If correlated, linear-ish graph vs lifespan, normal-ish distributed and continuous in R then we can apply linear regression</h3>
- Inputs: Smoking, Exercise<br>
- Output: Lifespan<br>

$ y=\alpha*x_{smoking}+\beta*x_{exercise}+c $

In [ ]:
from sklearn import linear_model 
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=0)

X = train[['smoking', 'exercise']]
y = train.lifespan
regr = linear_model.LinearRegression()
regr.fit(X, y) 

In [ ]:
# The coefficients
print('Coefficients: \n', regr.coef_)
print(f'c would be:',regr.predict([[0,0]]))

In regression, the $R^{2}$ coefficient of determination is a statistical measure of how well the regression predictions approximate the real data points.  <br>
Close to 1 means there is a strong correlation between the independent variables smoking, exercise and the dependent variable lifespan.

In [ ]:
print(train.shape, test.shape)
score = regr.score(test[['smoking', 'exercise']],test.lifespan)
print(f'coefficient of determination(R\N{SUPERSCRIPT TWO}):', score)

<h3>Step 6: Calculate the PremiumFactor for the insurance </h3><br>

$ PremiumFactor=\frac{genetic}{lifespan} $

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(color_codes=True)

Step 1. Get the data from API and make a dataframe

In [ ]:
# Make request to an URL
response = requests.get('http://localhost:8080/medish_centrum_randstad/api/netlify?page=1')

In [ ]:
file_contents= response.json()  #dictionary
print(type(file_contents))
print(len(file_contents))

In [ ]:
df = pd.DataFrame.from_dict(file_contents['data']) #all the needed info was condensed into one data column called 'data'
display(df.head())
display(df.shape)

In [ ]:
df= df[['genetic','length','mass','exercise','smoking','alcohol','sugar','lifespan']]
df.dropna(inplace=True)
print(df)

Step 3. Define BMI and Exploratory Data Analysis 

In [ ]:
#Define BMI
df['BMI'] = df['mass']/(df['length']/100)**2
display(df.head())

In [ ]:
#Why is it missing? Is it random?
#Impute or delete?
df = df.dropna()
df.head()

From boxplot we can check the outliers. I tried 2 ways to remove the outliers. One way is using IQR. The ohter way is using DBSCAN cluster. According to correlation gradient there were no differences in coreelation.  Therefore outliers does not make any difference. I decided to keep the outliers.

In [ ]:
sns.boxplot(y=df['exercise'])

#Q1=df['exercise'].quantile(0.25)
#print("Q1:", Q1)

#Q3=df['exercise'].quantile(0.75)
#print("Q3:", Q3)

#IQR=Q3-Q1
#print("IQR: ", IQR)

#lower_bound = Q1 - 1.5*IQR
#print("Lower Bound:", lower_bound)

#upper_bound = Q3 + 1.5*IQR
#print("Upper Bound:", upper_bound)

In [ ]:
from sklearn.cluster import DBSCAN

DBSCAN, which stands for density-based spatial clustering of applications with noise, is an unsupervised clustering algorithm. This approach identifies any points that are loosely packed or sit alone outside of densely packed clusters as outliers.

In [ ]:
X_train = df[['exercise','lifespan']]

In [ ]:
model = DBSCAN()
model.fit(X_train)

In [ ]:
cluster_labels = model.labels_
plt.scatter(df["exercise"], df["lifespan"], c = cluster_labels)
plt.show()

In [ ]:
df['labels'] = cluster_labels

In [ ]:
df_cluster_clean = df[df['labels'] != -1]

* Clean up outliers.

In [ ]:
sns.boxplot(data=df, y= 'lifespan', x='exercise')

In [ ]:
df_corr = df[['genetic',
      'length', 'mass', 
      'exercise', 'smoking',
       'alcohol', 'sugar', 'BMI','lifespan']].dropna().corr()
df_corr.style.background_gradient(cmap='RdBu')

lifespan is highly correlated to exercise. IQR method and DBSCAN cluster method showed same result.
Therefore we will remove outliers for a variabel,'exercise'.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from seaborn_qqplot import pplot

import matplotlib.pyplot as plt
%matplotlib inline     
sns.set(color_codes=True)

In [ ]:
df = pd.read_csv('data/new2.csv', index_col = 0)

In [ ]:
df

In [ ]:
# df.drop(indexes_list, inplace=True)
df = df.dropna().copy()
print(df)

In [ ]:
duplicateRows = df[df.duplicated()]
print(duplicateRows)

In [ ]:
# outliers er uit

In [ ]:
# bmi er bij

df['bmi'] = df['mass']/(df['length']/100)**2

In [ ]:
bmi_cats = [0, 18.5, 25, np.inf]
labels_bmi_cats=['underweight','normal_range','overweight']
df['bmi_cat']= pd.cut(df['bmi'], bins=bmi_cats, labels=labels_bmi_cats)

bmi_subcats = [0, 16, 17, 18.5, 25, 30, 35, 40, np.inf]
labels_bmi_subcats=['severe_thinness','moderate_thinness','mild_thinness','normal', 'pre_obese', 'obese_class_I', 'obese_class_II', 'obese_class_III']
df['bmi_subcat']= pd.cut(df['bmi'], bins=bmi_subcats, labels=labels_bmi_subcats)


df.head(12)

In [ ]:
# Maak een raster voor 4 images
import cv2

pImg = cv2.imread("pics/aiHealth_01.jpg")
# pImg = pplot(df, x="lifespan", y="alcohol", kind='qq', height=5)

fig, axs = plt.subplots(2, 2, sharey=False, figsize=(18,15))


axs[0][0].imshow(pImg)
pplot(df, x="lifespan", y="alcohol", kind='qq', height=5)

sns.boxplot(x=df['sugar'], ax=axs[1,1])

axs[0][1].scatter(df['sugar'], df['lifespan'])
axs[0][1].set_xlabel('sugar')
axs[0][1].set_ylabel('lifespan')

sns.catplot(data=df, x='sugar', y='lifespan', kind='box', ax=axs[1,1])

sns.lineplot(x=df['sugar'],y=df['lifespan'], ax=axs[1,0])




plt.show()

# print();

In [ ]:
sns.lineplot(df,x='sugar',y='lifespan')


In [ ]:
from seaborn_qqplot import pplot
pplot(df, x="lifespan", y="alcohol", kind='qq', height=5)

In [ ]:


plt.figure(figsize=(10,5))
c= df.corr()
sns.heatmap(c,cmap="BrBG",annot=True)
c